# Analysis Pipeline for Intracranial Data


## STEPS:
##### 1. run the IMPORT AND LOAD DATA section.

##### 2. fill in the values extracted from the plot from step 1 for following variables:


Let's go

In [1]:
import os
import numpy as np
import mne
from mne.utils import _TempDir
import matplotlib.pyplot as plt
import pyqtgraph as pg
import pyqtgraph as plotWidget
from get_trigger_indexes_photodiode import normalized
%matplotlib qt
from pd_parser.parse_pd import _read_raw, _to_tsv
import time
from get_trigger_indexes_photodiode import get_trigger_indexes_photodiode
from datetime import datetime
import read_trc
import struct
%matplotlib qt



# DEFINE PATHS
Exper = 'FLM'
PT_id = 'PAT_3390' #TODO
filename_edf = 'PAT_3390_EEG_860617_languageMapping.edf' #TODO
# PREDEFINED PATHS
data_path = '/Volumes/GoogleDrive/My Drive/EEG_DATA_PIERRE'
script_path = '/Volumes/GoogleDrive/My Drive/EEG_DATA_PIERRE/Scripts'
data_raw_file = os.path.join(data_path,PT_id, filename_edf)


# LOAD TRIGGER DATA 
print('\n\n*****----------------  ',Exper,'  ----------------*****\n' )
fig = plt.figure(figsize=(100, 2))
trig_chan_name = 'PHOTO'
t_onset_s= 0
t_offset_s= -1
index_onset,index_offset , trigger_ch_new,_ = get_trigger_indexes_photodiode(data_raw_file = data_raw_file, data_path=data_path,t_start= t_onset_s, t_end=t_offset_s,threshold_val=0.75, trigger_channel=trig_chan_name)

# FROM ABOVE PLOT, FILL THE TRIGGER ONSET OFFSETS PER EACH BLOCK
t_onset_s1= 400000 #TODO
t_offset_s1= 840000 #TODO
t_onset_s2= 970000 #TODO
t_offset_s2= 1680000 #TODO
t_onset_s3= 1690000 #TODO
t_offset_s3= 2500000 #TODO



<_io.BufferedReader name='/Volumes/GoogleDrive/My Drive/EEG_DATA_PIERRE/PAT_3066/PAT_3066_EEG_695997_Anon_FLM.TRC'>
212
['FIG1', 'FIG2', 'FIG3', 'FIG4', 'FIG5', 'FIG6', 'FIG7', 'FIG8', 'FIG9', 'FIG10', 'FIG11', 'FIG12', 'FOG1', 'FOG2', 'FOG3', 'FOG4', 'FOG5', 'FOG6', 'FOG7', 'FOG8', 'FOG9', 'FOG10', 'FOG11', 'FOG12', 'FOG13', 'FOG14', 'FOG15', 'CAG1', 'CAG2', 'CAG3', 'CAG4', 'CAG5', 'CAG6', 'CAG7', 'CAG8', 'CAG9', 'CAG10', 'CAG11', 'CAG12', 'IAG1', 'IAG2', 'IAG3', 'IAG4', 'IAG5', 'IAG6', 'IAG7', 'IAG8', 'IAG9', 'IAG10', 'IAG11', 'IAG12', 'IAG13', 'IAG14', 'IAG15', 'IAG16', 'IAG17', 'IAG18', 'MKR1+', 'IMG1', 'IMG2', 'IMG3', 'IMG4', 'IMG5', 'IMG6', 'IMG7', 'IMG8', 'IMG9', 'IMG10', 'IMG11', 'IMG12', 'IMG13', 'IMG14', 'IMG15', 'IMG16', 'IMG17', 'IMG18', 'AG1', 'AG2', 'AG3', 'AG4', 'AG5', 'AG6', 'AG7', 'AG8', 'AG9', 'AG10', 'HAG1', 'HAG2', 'HAG3', 'HAG4', 'HAG5', 'HAG6', 'HAG7', 'HAG8', 'HAG9', 'HAG10', 'HPG1', 'HPG2', 'HPG3', 'HPG4', 'HPG5', 'HPG6', 'HPG7', 'HPG8', 'HPG9', 'HPG10', 'HPG11'

objc[81742]: Class QCocoaPageLayoutDelegate is implemented in both /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/cv2/.dylibs/QtGui (0x12975a5c0) and /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/PyQt5/Qt/lib/QtPrintSupport.framework/Versions/5/QtPrintSupport (0x132629478). One of the two will be used. Which one is undefined.
objc[81742]: Class QCocoaPrintPanelDelegate is implemented in both /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/cv2/.dylibs/QtGui (0x12975a638) and /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/PyQt5/Qt/lib/QtPrintSupport.framework/Versions/5/QtPrintSupport (0x1326294f0). One of the two will be used. Which one is undefined.
objc[81742]: Class QCocoaApplicationDelegate is implemented in both /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/cv2/.dylibs/QtGui (0x12975a340) and /Library/Frameworks/Python.framework

*   Data location.... : /var/folders/85/ct7qgj4500s62pw9zxkv_xc40000gn/T/tmp_mne_tempdir_era6bets


Extracting EDF parameters from /Volumes/GoogleDrive/My Drive/EEG_DATA_PIERRE/PAT_3390/PAT_3390_EEG_860617_languageMapping.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
channels:  ['FOG1', 'FOG2', 'FOG3', 'FOG4', 'FOG5', 'FOG6', 'FOG7', 'FOG8', 'FOG9', 'FOG10', 'FOG11', 'FOG12', 'IAG1', 'IAG2', 'IAG3', 'IAG4', 'IAG5', 'IAG6', 'IAG7', 'IAG8', 'IAG9', 'IAG10', 'IAG11', 'IAG12', 'IAG13', 'IAG14', 'IAG15', 'IAG16', 'IAG17', 'IAG18', 'CAG1', 'CAG2', 'CAG3', 'CAG4', 'CAG5', 'CAG6', 'CAG7', 'CAG8', 'CAG9', 'CAG10', 'CAG11', 'CAG12', 'CPG1', 'CPG2', 'CPG3', 'CPG4', 'CPG5', 'CPG6', 'CPG7', 'CPG8', 'CPG9', 'CPG10', 'CPG11', 'CPG12', 'CPG13', 'CPG14', 'CPG15', 'EX1', 'EX2', 'EX3', 'EX4', 'EX5', 'EX6', 'EX7', 'MKR1+', 'TPG1', 'TPG2', 'TPG3', 'TPG4', 'TPG5', 'AG1', 'AG2', 'AG3', 'AG4', 'AG5', 'AG6', 'AG7', 'AG8', 'AG9', 'AG10', 'AG11', 'AG12', 'HAG1', 'HAG2'

### Extract the PD Trigger

In [2]:
from get_trigger_indexes_photodiode import get_trigger_indexes_photodiode
from datetime import datetime
%matplotlib qt

fig = plt.figure(figsize=(100, 2))
# # t_onset_s = input("Enter cut start time: ")
# # t_offset_s = input("Enter cut end time: ")
# # t_onset_s= 550000
# # t_offset_s= 3310000

Exper = 'FLM_image'
print('\n\n*****----------------  ',Exper,'  ----------------*****\n' )

t_onset_s= t_onset_s1
t_offset_s= t_offset_s1
index_onset,index_offset , trigger_ch_new,_ = get_trigger_indexes_photodiode(data_raw_file = data_raw_file, data_path=data_path,t_start= t_onset_s, t_end=t_offset_s,threshold_val=0.75,trigger_channel=trig_chan_name)

date_txt=str(datetime.date(datetime.now()))
name_txt = PT_id +Exper+'_triggerPD_' + date_txt+'.txt'
with open(os.path.join(data_path,name_txt ), "w") as textfile:
    textfile.write('onsets;offsets\n')
    XY = [i for i in zip(index_onset, index_offset)]
    for x, y in XY:
        textfile.write(str(x)+';'+str(y))
        textfile.write('\n')
    textfile.close()

#     # 28840
#     # 30446
# # index_on_true=index_onset
# # index_off_true=index_offset

Exper = 'FLM_audio'
print('\n\n*****----------------',Exper,'  ----------------*****\n' )
t_onset_s= t_onset_s2
t_offset_s= t_offset_s2
index_onset,index_offset , trigger_ch_new,_ = get_trigger_indexes_photodiode(data_raw_file = data_raw_file, data_path=data_path,t_start= t_onset_s, t_end=t_offset_s,threshold_val=0.75,trigger_channel=trig_chan_name)

date_txt=str(datetime.date(datetime.now()))
name_txt = PT_id +Exper+'_triggerPD_' + date_txt+'.txt'
with open(os.path.join(data_path,name_txt ), "w") as textfile:
    textfile.write('onsets;offsets\n')
    XY = [i for i in zip(index_onset, index_offset)]
    for x, y in XY:
        textfile.write(str(x)+';'+str(y))
        textfile.write('\n')
    textfile.close()

    # 28840
    # 30446
# index_on_true=index_onset
# index_off_true=index_offset



Exper = 'FLM_reading'
print('\n\n*****----------------',Exper,'  ----------------*****\n' )
t_onset_s= t_onset_s3
t_offset_s= t_offset_s3
index_onset,index_offset , trigger_ch_new,_ = get_trigger_indexes_photodiode(data_raw_file = data_raw_file, data_path=data_path,t_start= t_onset_s, t_end=t_offset_s,threshold_val=0.75,trigger_channel=trig_chan_name)

date_txt=str(datetime.date(datetime.now()))
name_txt = PT_id +Exper+'_triggerPD_' + date_txt+'.txt'
with open(os.path.join(data_path,name_txt ), "w") as textfile:
    textfile.write('onsets;offsets\n')
    XY = [i for i in zip(index_onset, index_offset)]
    for x, y in XY:
        textfile.write(str(x)+';'+str(y))
        textfile.write('\n')
    textfile.close()

    # 28840
    # 30446
# index_on_true=index_onset
# index_off_true=index_offset
# t_onset_s= 1022000
# t_offset_s= 1900000
# index_onset,index_offset , trigger_ch_new,_ = get_trigger_indexes_photodiode(data_raw_file = data_raw_file, data_path=data_path,t_start= t_onset_s, t_end=t_offset_s,threshold_val=0.75)

# index_on_true=index_onset
# index_off_true=index_offset

# t_onset_s= 1022000
# t_offset_s= 1900000
# index_onset,index_offset , trigger_ch_new,_ = get_trigger_indexes_photodiode(data_raw_file = data_raw_file, data_path=data_path,t_start= t_onset_s, t_end=t_offset_s,threshold_val=0.75)

# index_on_true=index_onset
# index_off_true=index_offset




*****----------------   FLM_image   ----------------*****

*   Data location.... : /var/folders/85/ct7qgj4500s62pw9zxkv_xc40000gn/T/tmp_mne_tempdir_cu8_i0j4


Extracting EDF parameters from /Volumes/GoogleDrive/My Drive/EEG_DATA_PIERRE/PAT_3390/PAT_3390_EEG_860617_languageMapping.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
channels:  ['FOG1', 'FOG2', 'FOG3', 'FOG4', 'FOG5', 'FOG6', 'FOG7', 'FOG8', 'FOG9', 'FOG10', 'FOG11', 'FOG12', 'IAG1', 'IAG2', 'IAG3', 'IAG4', 'IAG5', 'IAG6', 'IAG7', 'IAG8', 'IAG9', 'IAG10', 'IAG11', 'IAG12', 'IAG13', 'IAG14', 'IAG15', 'IAG16', 'IAG17', 'IAG18', 'CAG1', 'CAG2', 'CAG3', 'CAG4', 'CAG5', 'CAG6', 'CAG7', 'CAG8', 'CAG9', 'CAG10', 'CAG11', 'CAG12', 'CPG1', 'CPG2', 'CPG3', 'CPG4', 'CPG5', 'CPG6', 'CPG7', 'CPG8', 'CPG9', 'CPG10', 'CPG11', 'CPG12', 'CPG13', 'CPG14', 'CPG15', 'EX1', 'EX2', 'EX3', 'EX4', 'EX5', 'EX6', 'EX7', 'MKR1+', 'TPG1', 'TPG2', 'TPG3', 'TPG4', 'TPG5', 'AG1', 'AG2', 'AG3', 'AG4', 'AG5', 'AG6'

### VALIDATION
Assert that the trigger onset and offset are a pair and the onset is before the offset

In [3]:
# VALIDATION
# index_on_true=index_onset
# index_off_true=index_offset
# index_onset = list(index_onset)
# index_offset = list(index_offset)
# validity = [] # 0 for invalid, 1 for valid
# for i in range(min([len(index_on_true),len(index_off_true)])):
#     tempe=True
#     # print(i)
#     # if i >= min([len(index_on_true),len(index_off_true)]):
#     #     validity.append(0)
#     if index_onset[i] < index_offset[i]:
#         while tempe:
#             while index_onset[i] < index_offset[i]:
#                 if index_onset[i+1] <index_offset[i]:
#                     # print(index_onset[i])
#                     index_onset.pop(i)
#                     validity.append(0)
#                     tempe=False
#                     break
#                 else:
#                     validity.append(1)
#                     tempe=False
#                     break
#     else:
#         while tempe:
#             while index_onset[i] > index_offset[i]:
#                 if index_onset[i+1] >index_offset[i]:
#                     index_offset.pop(i)
#                     validity.append(0)
#                     tempte=False
#                 else:
#                     validity.append(1)
#                     tempe=False
#                     break



# if len(index_offset)!=len(index_onset):
#     if len(index_offset)>len(index_onset):
#         index_onset.append(0)
diff_oo=[index_offset[i]-index_onset[i] for i in range(len(index_offset))]
assert np.min(diff_oo)>=0
# print(validity, len(validity),len(index_onset),len(index_offset))
# validity=np.array(validity)

In [4]:

# VISUALIZE
fig = plt.figure(figsize=(100, 2))
print(np.shape(trigger_ch2))
import pyqtgraph as pg
import pyqtgraph as plotWidget
# pg.plot(time_s,trigger_ch2)
# pg.plot(time_s,index_onset)
# plt.close(fig)
time_s = np.linspace(0,len(trigger_ch2),num=len(trigger_ch2))
plotWidget = pg.plot(title="Validation of Triggers PD: x is onset, o is offset")
plotWidget.plot(time_s,trigger_ch2)
plotWidget.plot(index_offset,trigger_ch2[index_offset],pen=None, symbol='o')
plotWidget.plot(index_onset,trigger_ch2[index_onset],pen=None, symbol='x')
# plotWidget.plot(index_onset[validity==False],trigger_ch2[index_onset[validity==False]],pen=None, symbol='x')

# plotWidget.show()

(2181120,)


### Save The Data

In [6]:
from datetime import datetime
date_txt=str(datetime.date(datetime.now()))
name_txt = PT_id +Exper+'_triggerPD_' + date_txt+'.txt'
with open(os.path.join(data_path,name_txt ), "w") as textfile:
    textfile.write('onsets;offsets\n')
    XY = [i for i in zip(index_onset, index_offset)]
    for x, y in XY:
        textfile.write(str(x)+';'+str(y))
        textfile.write('\n')
    textfile.close()

    # 28840
    # 30446

### FINISHED HERE: 
##### Everything below is draft